In [4]:
import tensorflow as tf

In [10]:
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Helping Functions





In [12]:
# initialize weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [13]:
# initialize bias values
def init_bias(shape):
    init_bias_vals = tf.constant(value=0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [14]:
# 2D Convolution; convenience function to override tf's built-in function of the same name
def conv2d(x,w): 
    # x is input tensor with shape [batch,height H, width W, channels]
    # w is the kernel with shape [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(input=x,filter=w,strides=[1,1,1,1],padding='SAME') #SAME means pad w/ 0's

In [15]:
# pooling; convenience function to override tf's built-in function of the same name
def max_pool_2by2(x):
    # x is [batch,h,w,c]; c is color channel
    # ksize is size of window for each dimension of input tensor
    # strides is stride of sliding window for each dimension of input tensor
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

## Create the Layers

In [16]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x,shape):
    w = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,w)+b)

In [17]:
# NORMAL (FULLY CONNECTED) LAYER
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weights(shape=[input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,w) + b

In [18]:
#PLACEHOLDERS
x = tf.placeholder(tf.float32,shape=[None,784]) #None will become the batch size
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [35]:
#LAYERS
x_image = tf.reshape(tensor=x,shape=[-1,28,28,1]) #input; need to reshape flatened array into image

convo_1 = convolutional_layer(input_x=x_image,shape=[5,5,1,32]) #batch size 5x5, 1 input channel for grayscale, 32 features (output channels)
convo_1_pooling = max_pool_2by2(x=convo_1)

convo_2 = convolutional_layer(input_x=convo_1_pooling,shape=[5,5,32,64]) #build a deep network by stacking several layers
convo_2_pooling = max_pool_2by2(convo_2)

convo_2_flat = tf.reshape(tensor=convo_2_pooling,shape=[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [20]:
#DROPOUT (during training, units & their connections are randomly dropped)
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(x=full_layer_one,keep_prob=hold_prob)
y_pred = normal_full_layer(input_layer=full_one_dropout,size=10)

In [23]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [32]:
#OPTIMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001) #higher LR = faster, but less accurate
train = optimizer.minimize(cross_entropy)

In [33]:
init = tf.global_variables_initializer()

In [34]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50) #process 50 images at a time
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5}) #hold_prob is prob that a neuron won't be dropped 
        if i%100 == 0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY: ')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32)) #accuracy
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0})) #don't drop any neurons in test phase
            print('\n')

ON STEP: 0
ACCURACY: 
0.111


ON STEP: 100
ACCURACY: 
0.9425


ON STEP: 200
ACCURACY: 
0.964


ON STEP: 300
ACCURACY: 
0.9708


ON STEP: 400
ACCURACY: 
0.976


ON STEP: 500
ACCURACY: 
0.9763


ON STEP: 600
ACCURACY: 
0.9777


ON STEP: 700
ACCURACY: 
0.9802


ON STEP: 800
ACCURACY: 
0.9799


ON STEP: 900
ACCURACY: 
0.985


ON STEP: 1000
ACCURACY: 
0.9806


ON STEP: 1100
ACCURACY: 
0.9877


ON STEP: 1200
ACCURACY: 
0.9849


ON STEP: 1300
ACCURACY: 
0.988


ON STEP: 1400
ACCURACY: 
0.9893


ON STEP: 1500
ACCURACY: 
0.9871


ON STEP: 1600
ACCURACY: 
0.9871


ON STEP: 1700
ACCURACY: 
0.9879


ON STEP: 1800
ACCURACY: 
0.9885


ON STEP: 1900
ACCURACY: 
0.986


ON STEP: 2000
ACCURACY: 
0.9887


ON STEP: 2100
ACCURACY: 
0.9889


ON STEP: 2200
ACCURACY: 
0.9896


ON STEP: 2300
ACCURACY: 
0.9887


ON STEP: 2400
ACCURACY: 
0.9904


ON STEP: 2500
ACCURACY: 
0.9901


ON STEP: 2600
ACCURACY: 
0.9886


ON STEP: 2700
ACCURACY: 
0.9877


ON STEP: 2800
ACCURACY: 
0.9903


ON STEP: 2900
ACCURACY: 
0.9886
